### Press Ctrl + A on the Output tab of the epiic webpage, and copy everything
### Then copy it into a output.txt in this directory, the notebook below will parse it

In [440]:
import re

In [470]:
from collections import namedtuple

VHDLOutput = namedtuple("VHDLOutput", ["time", 
                                       "instruction", "pc", 
                                       "aluA", "aluB", "aluResult", 
                                       "result", "destReg", 
                                       "regA", "regB", "regAddrA", "regAddrB",
                                       "jumpAddr", "addrSel", "nextAddr", "branchAddr",
                                       "regwrite", "regdst", "alusrc", "branch", "memwrite", "memtoreg", "jump", "aluCtrl"])

In [471]:
def b2hex(s):
    if "-" in s:
        return s
    return "0x{:08x}".format(int(s, 2))

def b2int(s):
    if "-" in s:
        return s
    return int(s, 2)

def b2debug(s):
    print("bin:", s)
    print("hex: ", b2hex(s))
    print("int: ", b2int(s))

In [730]:
def parse_messages(lines):
    messages = [[]]
    # group lines between Begin and End
    curr_message = messages[0]
    for l in lines:
        if "Begin" in l:
            curr_message.clear()
        curr_message.append(l)
        if "End" in l:
            curr_message = []
            messages.append(curr_message)
    
    outputs = []
    for message in messages[1:]:
        if(len(message) < 2):
            continue
        output = message[2:-1:2]  # remove Begin and End line as well as the timers between
        time = int(re.findall(r'\b\d+\b', message[1])[0])
        all_numbers = [re.findall(r'[01\-]+', o) for o in output]
        try:
            o = VHDLOutput(time,
                *all_numbers[0],
                *all_numbers[1],
                *all_numbers[2],
                *all_numbers[3],
                *all_numbers[4],
                *all_numbers[5]
            )
            outputs.append(o)
        except Exception as e:
            print(e)
            print("\n".join(message))
        
    return outputs

In [731]:
with open("output.txt") as file:
    lines = [l for l in file.read().split("\n") if l[:9] == "# ** Note" or l[:9] == "#    Time"]
    lines = [l for l in lines if "Program terminated" not in l]

outputs = parse_messages(lines)

list index out of range
#    Time: 400 ps  Iteration: 0  Instance: /testbench/dut/dp
#    Time: 405 ps  Iteration: 1  Instance: /testbench


### Assert some values

In [732]:
expected_values = {
    2: {
        "pc": 0,
    },
    3: {
        "pc": 4,
        "jumpAddr": 20
    },
    4: {   # addi $29, $29, -4
        "pc": 20,   # a jump took place
        "instruction": 0x23bdfffc,
        "aluA": 512,
        "aluResult": 508
    },
    5: {
        "pc": 24,
        "instruction": 0xafbe0000
    },
    6: {
        "pc": 28
    },
    7: {
        "pc": 32
    },
    8: {
        "pc": 36,
        "instruction": 0x20090001
    },
    9: {
        "pc": 40,
        "result": 32   # space
    },
    10: {
        "pc": 44,
        "branch": 1,
        "addrSel": 0,
        "nextAddr": 48,
        "branchAddr": 120
    },
    11: {
        "pc": 48,
        "branch": 1,
        "addrSel": 0,
        "nextAddr": 52,
        "branchAddr": 84
    },
    12: {
        "pc": 52,
        "aluCtrl": 7, # sub
        "result": 1
    },
    13: {
        "pc": 56,  # bne $t3, $t0, else1
        "aluCtrl": 6,
        "addrSel": 1,
    },
    14: {
        "pc": 84,  # a jump happened,
        "result": 32
    },
    15: {
        "pc": 88,
        "aluA": 32,
        "aluB": 32,
        "addrSel": 0   # no jump, since they are equal and we have `bne`
    },
    16: {
        "pc": 92,
        "aluA": 0,
        "aluB": 1,
        "aluResult": 1,
        "result": 1
    },
    17: {
        "pc": 96,
        "result": 0  # sub 1 - 1 = 0
    },
    19: {
        "pc": 104
    },
    20: {
        "pc": 112
    },
    21: {
        "pc": 116 ## this is a jump to 40
    },
    22: {
        "pc": 40,
        "result": 116  # lower case t
    },
    29: {
        "pc": 68,
        "aluA": 116,
        "aluResult": 116 - 32
    },
    30: {
        
    }
    
}

for k, values in expected_values.items():
    actual_o = outputs[k]
    for key, int_val in values.items():
        assert int(getattr(actual_o, key), 2) == int_val, f"Error: at index {k}: {key} expected value {int_val} but was {int(getattr(actual_o, key), 2)}"

bin: 01101001011010000111010000100000

hex:  0x69687420

int:  1768453152


In [744]:
o = outputs[33]
b2debug(o.pc)

bin: 00000000000000000000000001111000
hex:  0x00000078
int:  120


In [747]:
b2hex("00001000000000000000000000011100")

'0x0800001c'